In [1]:
import os
import openai
import sys
sys.path.append('../..')

import constants
openai.api_key = constants.OPENAI_API_KEY

In [3]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
persist_directory = 'docs/chroma/'
embedding = OpenAIEmbeddings(openai_api_key=constants.OPENAI_API_KEY)
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [4]:
print(vectordb._collection.count())

104


In [5]:
question = "Wie arbeiten wir mit TIS?"
docs = vectordb.similarity_search(question,k=3)

In [6]:
len(docs)

3

In [8]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(openai_api_key=constants.OPENAI_API_KEY, model_name="gpt-3.5-turbo", temperature=0)

In [9]:
from langchain.chains import RetrievalQA

In [10]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever = vectordb.as_retriever()
)

In [11]:
result = qa_chain({"query":question})

In [12]:
result["result"]

'Wir arbeiten mit TIS, indem wir unsere UBS Konten mit dem Treasury Intelligence Solution (TIS) Programm verbinden. Dadurch werden alle Transaktionen analog zu unseren UBS Konten auch in TIS angezeigt. Über TIS können wir auch UBS Banktransaktionen ausführen. Für die Verbuchung von Zahlungseingängen nutzen wir den direkten Download des elektronischen Kontoauszugs via TIS in unser SAP System. Weitere Informationen zu TIS findest du unter (https://www.tis.biz/).'

In [13]:
from langchain.prompts import PromptTemplate

template = """use the following pieces of context to answer the question
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [14]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever = vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [15]:
question = "Bis wann ist das Postkonto aufzulösen?"

In [16]:
result = qa_chain({"query": question})

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).


In [17]:
result["result"]

'Das Postkonto soll bis Ende 2017 aufgelöst werden.'

In [19]:
result["source_documents"][0]

Document(page_content='3 1 Einleitung  Hintergrund: In Zukunft soll nur noch mit der UBS gearbeitet werden. Folglich ist geplant das Postkonto bis Ende 2017 aufzulösen. Hierzu müssen wir die Kunden dringend informieren, dass zukünftig nur noch unsere UBS Konten Gültigkeit haben. 28.12.2017: All diese Aktivitäten sind erfolgt. Unsere Post Verbindung ist gekündigt und wir arbeiten fortan nur noch mit UBS.  Exkurs TIS: Unsere UBS Konten sind mit unserem Treasury Intelligence Solution (TIS) Programm verbunden. Das heisst alle Transaktionen werden analog zu unseren UBS Konten auch in TIS angezeigt. Weiter können UBS Banktransaktionen in TIS ausgeführt werden. Für die Verbuchung von Zahlungseingängen via TIS bedeutet das den direkten Download des sogenannten elektronischen Kontoauszugs via TIS in unser SAP System. Weiter können über TIS zahlreiche UBS Banktransaktionen ausgeführt werden. Weitere Infos siehe (https://www.tis.biz/).  Effizienzhinweis vorab:  Diese Favoriten werden benötigt. Es

In [20]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [21]:
question = "Was muss ich tun, wenn die Zahlungen im FEBAN verbucht sind?"
result = qa_chain({"query": question})
result["result"]

'Wenn die Zahlungen im FEBAN verbucht sind, müssen Sie die Zahlungen ausziffern. Dafür öffnen Sie die Transaktion FEBAN und nehmen einige einmalige Einstellungen vor. Anschließend bestätigen Sie die Einstellungen und können die Zahlungen ausziffern.'

In [22]:
question = "Wo kann ich die eletkronischen Kontoauszüge abrufen?"
result = qa_chain({"query": question})
result["result"]

'Die elektronischen Kontoauszüge können jeweils direkt in SAP abgerufen werden.'